In [1]:
# Installation of any non native supported libraries

# !pip install beautifulsoup4
# !pip install wget

In [2]:
# Import necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import os
import re 
import openpyxl
import pandas as pd
from datetime import datetime
import calendar
from pyjstat import pyjstat

In [3]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

## Download Data

In [4]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

### IE Monthly Statistics of Building Materials and Components

In [148]:
EXAMPLE_URL = 'https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/WPM28/JSON-stat/2.0/en/'

# read from json-stat
dataset = pyjstat.Dataset.read(EXAMPLE_URL)

# write to dataframe
df = dataset.write('dataframe')
df = df[df['Statistic'] == 'Wholesale Price Index (Excl VAT) for Building and Construction Materials']
ie_df = df.drop(['Statistic'], axis=1)
ie_df = ie_df.groupby(['Type of Material','Month'], sort=False)['value'].mean().unstack()
ie_df.to_csv(download_destination + '/IE Data/IE_Building_Materials_CostIndex_Compiled.csv')

### UK Monthly Statistics of Building Materials and Components

In [179]:
Downloaded_data = []

# Current Date
current_date = datetime.today()
current_year, current_month = int(current_date.strftime('%Y')), int(current_date.strftime('%m'))
current_month = current_month - 2 # Account for delay uploading data by data.gov.uk
start_year = current_year


while current_year >= 2018:
    # Webpage
    URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, "html.parser")
    file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    
    for a in soup.find_all("a", href=lambda href: href.lower() and str(current_year).lower() in href and str(calendar.month_name[current_month]).lower() in href.lower()):
        l = a['href']
        
    if not l.endswith('.xlsx'):
        r = requests.get(l) 
        soup = BeautifulSoup(r.content, "html.parser")
        for a in soup.find_all("a", href=lambda href: href.lower() and str(current_year).lower() in href and str(calendar.month_name[current_month]).lower() in href.lower() and '.xlsm' in href.lower(), limit=1):
            l = a['href']
            
    # Download link
    print(l)
    r = requests.get(l) 
    with open(file_name, 'wb') as f:
        f.write(r.content)     
    Downloaded_data.append(file_name)
    
    current_year = current_year - 1
    
print('')
print('100% |■■■■■■■■■■■■■■■■■■■■■■■■■■■|')
print('Total files downloaded: ' + str(len(Downloaded_data)))

2023 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx
2022 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx
2021 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/976157/21-cs4_-_Construction_Building_Materials_-_Tables_March_2021.xlsm
2020 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/876765/20-cs4_-_Construction_Building_Materials_-_Tables_March_2020.xlsm
2019 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/791320/19-cs4_-_Construction_Building_Materials_-_Excel_Tables_March_2019.xlsm
2018 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachm

In [653]:
# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [654]:
current_year = 2023
for wb in Downloaded_data:
    print('{} {}'.format(current_year, calendar.month_name[current_month]))

    wb = openpyxl.load_workbook(wb)
    sheets = wb.sheetnames
    ws = wb[sheets[3]]
    df = pd.DataFrame(ws.values)

    df = df.T.set_index(6).T 
    df.columns.values[0] = 'Materials'
    df = df.drop([col for col in df.columns.values if not col], axis=1) # Delete Filler Columns
    mat = df.loc[df.iloc[:,0].values != None]
    mat = mat['Materials'].values
    mat = mat[4:49]
    mat = [w for w in mat if not w.isupper() or w.isdigit()]
    df = df[df['Materials'].isin(mat)]
    df.reset_index(drop=True, inplace=True) # Reset the index 
    df.columns.name = 'Index'
    if current_year <= 2020:
        rows_to_delete = [0,1]
        df = df.drop(index=rows_to_delete)
        df1 = df.copy()
        df.iloc[0,:],df.iloc[1,:]= df1.iloc[1,:],df1.iloc[0,:]
        df.reset_index(drop=True, inplace=True) # Reset the index

    curr_month = current_month
    curr_year = current_year
    col_list = [col for col in df.columns[::-1]]
    new_col = []
    for c in col_list[:-1]:
        curr_month = curr_month - 1
        if curr_month == 0:
            curr_year = curr_year - 1
            curr_month = 12  
        c = str(curr_year) + ' ' + calendar.month_name[curr_month]
        new_col.append(c)    
    new_col.append('Materials') 
    new_col.reverse()
    df.columns = new_col
    
    if current_year == 2023:
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        new_df = pd.merge(df, UK_Data_Masterfile, left_index=True, right_index=True, how='outer')
        new_df.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    else:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        new_df = pd.merge(df, UK_Data_Masterfile, left_index=True, right_index=True, how='inner', suffixes=('', '_y'))
        new_df = new_df.drop(new_df.filter(regex='_y$').columns, axis=1)
        new_df.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    current_year = current_year - 1

2023 March
2022 March
2021 March
2020 March
2019 March
2018 March
